In [18]:
class CFG:
    device = "cuda" if torch.cuda.is_available() else "cpu"
    epochs = 7
    lr = 0.001


In [19]:
#################################
# efficientnet_b0
#################################
from torch import nn
import torchvision
import numpy as np
class DateClassifier(nn.Module):
    def __init__(self, num_classes=9 , pretrained=True):
        super(DateClassifier, self).__init__()
        self.model = torchvision.models.efficientnet_b0(pretrained=pretrained)
        self.model.classifier = nn.Sequential(
            nn.Linear(self.model.classifier[1].in_features, 512),
            nn.BatchNorm1d(512),  # Batch Normalization added here
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes),
            nn.BatchNorm1d(num_classes),  # Batch Normalization added here
        )

    def forward(self, x):
        return self.model(x)


In [20]:
!pip install gradio wikipedia

import gradio as gr
import wikipedia
from PIL import Image
import torch
import torchvision.transforms as transforms

# Define the classes (assuming these are the same as used in your model)
classes = ['Ajwa', 'Galaxy', 'Medjool', 'Meneifi', 'Nabtat Ali', 'Rutab', 'Shaishe', 'Sokari', 'Sugaey']

# Load the saved model state dictionary
# Ensure 'best_model.pth' exists and is the path to your trained model
model_path = 'best_model (2).pth'
try:
    # Create a new instance of the model architecture
    model = DateClassifier(num_classes=len(classes), pretrained=False).to(CFG.device) # Set pretrained=False when loading weights

    # Load the state dictionary
    model.load_state_dict(torch.load(model_path, map_location=torch.device(CFG.device)))
    model.eval() # Set model to evaluation mode
    print(f"Model loaded successfully from {model_path}")
except FileNotFoundError:
    print(f"Error: Model file not found at {model_path}. Please ensure the training and saving process completed successfully.")
    model = None # Set model to None if loading fails

# Define the image transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    # Add normalization if your training data was normalized
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def classify_date_and_get_description(image):
    if model is None:
        return "Model not loaded.", "Please ensure the model file exists and is correctly loaded."

    try:
        # Convert the Gradio Image to a PIL Image if needed
        if isinstance(image, np.ndarray):
            image = Image.fromarray(image.astype('uint8'), 'RGB')

        # Apply transformations
        image = transform(image).unsqueeze(0).to(CFG.device) # Add batch dimension

        # Make a prediction
        with torch.no_grad():
            outputs = model(image)
            _, predicted_idx = torch.max(outputs, 1)
            predicted_class = classes[predicted_idx.item()]

        # Get description from Wikipedia
        try:
            # Search Wikipedia for the date type
            page = wikipedia.page(f"{predicted_class} date", auto_suggest=True)
            description = page.summary
        except wikipedia.exceptions.DisambiguationError as e:
            description = f"Wikipedia disambiguation for '{predicted_class} date': {e}"
        except wikipedia.exceptions.PageError:
            description = f"Could not find a specific Wikipedia page for '{predicted_class} date'. Searching for just '{predicted_class}'."
            try:
                 page = wikipedia.page(f"{predicted_class}", auto_suggest=True)
                 description = page.summary
            except:
                 description = f"Could not find a relevant Wikipedia page for '{predicted_class}'."
        except Exception as e:
            description = f"An error occurred while fetching Wikipedia information: {e}"


        return predicted_class, description

    except Exception as e:
        return f"Error during processing: {e}", "Could not classify the image or fetch description."


# Create the Gradio interface
if model is not None:
    interface = gr.Interface(
        fn=classify_date_and_get_description,
        inputs=gr.Image(type="numpy", label="Upload Date Image"),
        outputs=[
            gr.Textbox(label="Date Type"),
            gr.Textbox(label="Description (from Wikipedia)")
        ],
        title="Date Fruit Classifier",
        description="Upload an image of a date fruit and I'll tell you its type and provide a brief description from Wikipedia."
    )

    # Launch the interface
    interface.launch(debug=True)
else:
    print("Gradio interface cannot be launched because the model failed to load.")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Model loaded successfully from best_model (2).pth
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://57cfb8d91eb760aadd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.11/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.11/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://154c7480472ebf23dc.gradio.live
Killing tunnel 127.0.0.1:7861 <> https://57cfb8d91eb760aadd.gradio.live
